# 1. Descrição do problema de negócio
..

# 2. Carregando dados

## 2.1 Importando dependências

Importando as bibliotecas necessárias para as etapas inicias de carregamento de dados, exploração e transformação.

In [1]:
## Load dependencies.
import pandas as pd
import json
import requests
import numpy as np
from global_functions import strip_upper
from global_functions import map_values_caged
from global_functions import map_values_general
from global_functions import quote
from global_functions import melt_pib
from global_functions import search_info

## 2.1 Carregando dados através da API CKAN.

A base de dados carregada aqui é referente as operações de financiamento não automáticas dos dados abertos do BNDES.

In [2]:
## Define resource identifier and server to create our URL string for an API request.
res_id = '332d446e-d340-46ef-af64-ee6f36e7bd50' ## Operações de financiamento não automáticas.
server = 'https://dadosabertos.bndes.gov.br'

## Define query limit.
limit = 20000

## Define filters.
filters = quote({'setor_bndes': ['INFRAESTRUTURA', 'INDUSTRIA', 'AGROPECUÁRIA']})
                 
## Define fields to query.
fields = 'cliente,cnpj,uf,data_da_contratacao,valor_contratado_reais, \
            valor_desembolsado_reais,fonte_de_recurso_desembolsos,custo_financeiro, \
            juros,prazo_carencia_meses,prazo_amortizacao_meses,modalidade_de_apoio,produto, \
            instrumento_financeiro,inovacao,area_operacional,setor_bndes,subsetor_bndes, \
            porte_do_cliente,natureza_do_cliente,tipo_de_garantia,situacao_do_contrato'

## Create query url.
query = (f"{server}/api/3/action/datastore_search?resource_id={res_id}"
           f"&filters={filters}&fields={fields}&limit={limit}")

print('Url para consulta:', query)


Url para consulta: https://dadosabertos.bndes.gov.br/api/3/action/datastore_search?resource_id=332d446e-d340-46ef-af64-ee6f36e7bd50&filters=%7B%22setor_bndes%22%3A%20%5B%22INFRAESTRUTURA%22%2C%20%22INDUSTRIA%22%2C%20%22AGROPECU%5Cu00c1RIA%22%5D%7D&fields=cliente,cnpj,uf,data_da_contratacao,valor_contratado_reais,             valor_desembolsado_reais,fonte_de_recurso_desembolsos,custo_financeiro,             juros,prazo_carencia_meses,prazo_amortizacao_meses,modalidade_de_apoio,produto,             instrumento_financeiro,inovacao,area_operacional,setor_bndes,subsetor_bndes,             porte_do_cliente,natureza_do_cliente,tipo_de_garantia,situacao_do_contrato&limit=20000


In [3]:
### NOT USED ###
## Get number of observations of the dataset.
##try:
  ##  data = requests.get(query)
    #if data.json()['success'] == True:
     #   limit = data.json()['result']['total']

    #else:
     #   print(data.json()['error'])
#except:
 #   data.raise_for_status()
# big_query = (f"{query}&limit={limit}")


In [4]:
## Request dataset.
try:
    data = requests.get(query)
    if data.json()['success'] == True:
        df_fin = pd.DataFrame(data.json()['result']['records'])

    else:
        print(data.json()['error'])
except:
    data.raise_for_status()


# 3. Data wrangling - Transformando e mapeando os dados para análise.


## 3.1 Explorando o dataset 'Operações não automáticas'

In [5]:
## Overview of the dataset.
df_fin.head()

,cliente,cnpj,uf,data_da_contratacao,valor_contratado_reais,valor_desembolsado_reais,fonte_de_recurso_desembolsos,custo_financeiro,juros,prazo_carencia_meses,...,produto,instrumento_financeiro,inovacao,area_operacional,setor_bndes,subsetor_bndes,porte_do_cliente,natureza_do_cliente,tipo_de_garantia,situacao_do_contrato
0,ELETROPAULO METROPOLITANA ELETRICIDADE DE SAO ...,61.695.227/0001-93,SP,2002-02-04T00:00:00,1.820297e+08,1.820297e+08,RECURSOS LIVRES - TESOURO,TAXA FIXA,1.0,63,...,BNDES FINEM,OUTROS,NÃO,AREA DE ENERGIA,INFRAESTRUTURA,ENERGIA ELÉTRICA,GRANDE,PRIVADA,REAL,LIQUIDADO
1,VIDEOLAR-INNOVA S/A,04.229.761/0001-70,AM,2002-01-10T00:00:00,5.000000e+05,5.339624e+05,RECURSOS LIVRES - FAT / RECURSOS LIVRES - PRÓP...,TJLP,1.0,24,...,BNDES FINEM,OUTROS,NÃO,AREA DE INDUSTRIA E SERVICOS,INDUSTRIA,MECÂNICA,GRANDE,PRIVADA,REAL / PESSOAL,LIQUIDADO
2,VIDEOLAR-INNOVA S/A,04.229.761/0001-70,AM,2002-01-10T00:00:00,5.714000e+06,5.340896e+06,RECURSOS LIVRES - FAT / RECURSOS LIVRES - PRÓP...,US$ / CESTA,4.5,12,...,BNDES FINEM,OUTROS,NÃO,AREA DE INDUSTRIA E SERVICOS,INDUSTRIA,MECÂNICA,GRANDE,PRIVADA,REAL / PESSOAL,LIQUIDADO
3,VIDEOLAR-INNOVA S/A,04.229.761/0001-70,AM,2002-01-10T00:00:00,2.968000e+07,2.999406e+07,RECURSOS LIVRES - FAT / RECURSOS LIVRES - PRÓP...,TJLP,4.5,12,...,BNDES FINEM,OUTROS,NÃO,AREA DE INDUSTRIA E SERVICOS,INDUSTRIA,MECÂNICA,GRANDE,PRIVADA,REAL / PESSOAL,LIQUIDADO
4,USINA CAETE S A,12.282.034/0001-03,AL,2002-01-11T00:00:00,6.323444e+06,6.408000e+06,RECURSOS LIVRES - PRÓPRIOS / RECURSOS VINCULAD...,TJLP,3.5,30,...,BNDES FINEM,OUTROS,NÃO,AREA DE INDUSTRIA E SERVICOS,INFRAESTRUTURA,ENERGIA ELÉTRICA,GRANDE,PRIVADA,REAL / PESSOAL,LIQUIDADO


In [6]:
## Dataset dimensions
df_fin.shape

(15496, 22)

In [7]:
## Veryfing duplicated records
df_fin.duplicated().sum()

565

In [8]:
## Veryfing NA's
df_fin.isna().sum()

cliente                         0
cnpj                            0
uf                              0
data_da_contratacao             0
valor_contratado_reais          0
valor_desembolsado_reais        0
fonte_de_recurso_desembolsos    0
custo_financeiro                0
juros                           0
prazo_carencia_meses            0
prazo_amortizacao_meses         0
modalidade_de_apoio             0
produto                         0
instrumento_financeiro          0
inovacao                        0
area_operacional                0
setor_bndes                     0
subsetor_bndes                  0
porte_do_cliente                0
natureza_do_cliente             0
tipo_de_garantia                0
situacao_do_contrato            0
dtype: int64

In [9]:
## Printing unique bndes sector
print(df_fin['setor_bndes'].unique())

## Printing unique bndes sub sector
print(df_fin['subsetor_bndes'].unique())


['INFRAESTRUTURA' 'INDUSTRIA' 'AGROPECUÁRIA']
['ENERGIA ELÉTRICA' 'MECÂNICA' 'MATERIAL DE TRANSPORTE' 'AGROPECUÁRIA'
 'OUTROS TRANSPORTES' 'TÊXTIL E VESTUÁRIO' 'OUTRAS' 'ALIMENTO E BEBIDA'
 'METALURGIA E PRODUTOS' 'CONSTRUÇÃO' 'QUÍMICA E PETROQUÍMICA'
 'TRANSPORTE RODOVIÁRIO' 'SERV. UTILIDADE PÚBLICA' 'ATV. AUX. TRANSPORTES'
 'EXTRATIVA' 'TRANSPORTE FERROVIÁRIO' 'CELULOSE E PAPEL'
 'TELECOMUNICAÇÕES']


In [10]:
## Subsectors of AGROPECUÁRIA
df_fin.loc[df_fin['setor_bndes'] == 'AGROPECUÁRIA']['subsetor_bndes'].unique()

array(['AGROPECUÁRIA'], dtype=object)

In [11]:
## Subsectors of INFRAESTRUTURA
df_fin.loc[df_fin['setor_bndes'] == 'INFRAESTRUTURA']['subsetor_bndes'].unique()


array(['ENERGIA ELÉTRICA', 'OUTROS TRANSPORTES', 'CONSTRUÇÃO',
       'TRANSPORTE RODOVIÁRIO', 'SERV. UTILIDADE PÚBLICA',
       'ATV. AUX. TRANSPORTES', 'TRANSPORTE FERROVIÁRIO',
       'TELECOMUNICAÇÕES'], dtype=object)

In [12]:
## Subsectors of INDUSTRIA
df_fin.loc[df_fin['setor_bndes'] == 'INDUSTRIA']['subsetor_bndes'].unique()


array(['MECÂNICA', 'MATERIAL DE TRANSPORTE', 'TÊXTIL E VESTUÁRIO',
       'OUTRAS', 'ALIMENTO E BEBIDA', 'METALURGIA E PRODUTOS',
       'QUÍMICA E PETROQUÍMICA', 'EXTRATIVA', 'CELULOSE E PAPEL'],
      dtype=object)

## 3.2 Padronizando a formatação dos dados

In [13]:
# Indexing a new column with year format
df_fin['ano'] = pd.to_datetime(df_fin['data_da_contratacao'])
df_fin['ano'] = pd.DatetimeIndex(df_fin['ano']).year

In [14]:
## Strip setor and subsetor BNDES from special characters
df_fin['setor_bndes'] = strip_upper(df_fin['setor_bndes'].tolist(),'')
df_fin['subsetor_bndes'] = strip_upper(df_fin['subsetor_bndes'].tolist(), '')

## Printing unique bndes sector
print(df_fin['setor_bndes'].unique())

## Printing unique bndes sub sector
print(df_fin['subsetor_bndes'].unique())

['INFRAESTRUTURA' 'INDUSTRIA' 'AGROPECUARIA']
['ENERGIA ELETRICA' 'MECANICA' 'MATERIAL DE TRANSPORTE' 'AGROPECUARIA'
 'OUTROS TRANSPORTES' 'TEXTIL E VESTUARIO' 'OUTRAS' 'ALIMENTO E BEBIDA'
 'METALURGIA E PRODUTOS' 'CONSTRUCAO' 'QUIMICA E PETROQUIMICA'
 'TRANSPORTE RODOVIARIO' 'SERV. UTILIDADE PUBLICA' 'ATV. AUX. TRANSPORTES'
 'EXTRATIVA' 'TRANSPORTE FERROVIARIO' 'CELULOSE E PAPEL'
 'TELECOMUNICACOES']


## 3.3 - Adicionando bases externas: CAGED, SEEG e PIB (ano/uf/setor)

In [15]:
## Loading datasets.
caged = pd.read_csv('bases-auxiliares/CAGED-ajustado.csv')
seeg_emissao = pd.read_csv('bases-auxiliares/SEEG-emissao-ajustado.csv')
pib_ind = pd.read_csv('bases-auxiliares/PIB-IND.csv')
pib_agro = pd.read_csv('bases-auxiliares/PIB-agro.csv')


In [16]:
## Append number of employees from CAGED dataset, segmented by company_size/sector/uf/ano
df_fin = map_values_caged(df_fin, caged, 'setor_bndes', 'num_empregados')

In [17]:
## Append Co2 emissions information from SEEG dataset, segmented by sector/uf/ano
df_fin = map_values_general(df_fin, seeg_emissao, 'setor_bndes', 'emissao_co2')

In [18]:
## Set columns to drop from PIB datasets
col_drops = ['Código', 'Estado']

## Change PIB datasets structure to match our project standards
pib_agro = melt_pib(pib_agro, col_drops, 'AGROPECUARIA')
pib_ind = melt_pib(pib_ind, col_drops, 'INDUSTRIA')

In [19]:
## Append PIB datasets
pib = pd.concat([pib_agro, pib_ind], axis = 0, ignore_index = True)
print(pib.shape)
print(pib.head())
print(pib['setor'].unique())

(864, 4)
   uf   ano         setor           pib
0  AC  2002  AGROPECUARIA  5.401684e+05
1  AL  2002  AGROPECUARIA  4.700566e+06
2  AM  2002  AGROPECUARIA  2.354165e+06
3  AP  2002  AGROPECUARIA  7.825019e+04
4  BA  2002  AGROPECUARIA  1.256136e+07
['AGROPECUARIA' 'INDUSTRIA']


In [20]:
## Set format to numeric
pib['ano'] = pd.to_numeric(pib['ano'])

In [21]:
## Append PIB information, segmented by sector/uf/ano
df_fin = map_values_general(df_fin, pib, 'setor_bndes', 'pib')

In [23]:
## Round numeric values
toRound = ['valor_contratado_reais', 'valor_desembolsado_reais', 'num_empregados', 'emissao_co2', 'pib']
df_fin[toRound] = df_fin[toRound].round(1)

In [24]:
## Exporting integrated dataset
df_fin.to_csv('bases-auxiliares/operacoes-integrado.csv',
              index=False, header=True)


## 3.4 - Agrupamento
A granularidade dos dados colhidos está no nível de ano - uf - setor_bndes e porte do cliente. Devido a isso, nesta etapa o dataset será agrupado para representar este formato e permitir análises estatísticas nesses níveis.

### 3.4.1 - Agrupamento para os dados CAGED
Os dados do CAGED tem um nível de granularidade maior do que os dados do SEEG e do PIB, devido a presença da segmentação por porte do cliente. Devido a isso, o primeiro agrupamento será para este formato.

In [25]:
## Drop columns with higher segmentation levels
drop_cols = ['cliente', 'cnpj', 'data_da_contratacao', 'fonte_de_recurso_desembolsos',
                'custo_financeiro', 'modalidade_de_apoio', 'produto', 'instrumento_financeiro',
                'inovacao', 'area_operacional', 'subsetor_bndes', 'natureza_do_cliente', 'tipo_de_garantia',
                'situacao_do_contrato']
df_fin.drop(drop_cols, axis=1, inplace=True)


In [26]:
## Group dataset with mean values and by uf, setor_bndes, porte_do_cliente and ano
df_grouped = df_fin.groupby(['uf', 'setor_bndes', 'porte_do_cliente', 'ano'], as_index=False).agg(
    {'valor_contratado_reais': ['mean'], 'valor_desembolsado_reais': ['mean'],
    'juros': ['mean'], 'prazo_carencia_meses': ['mean'], 'prazo_amortizacao_meses': ['mean'],
    'num_empregados': ['mean'], 'emissao_co2': ['mean'], 'pib': ['mean']})

In [27]:
## Melt multi index from pandas
df_grouped.columns = list(map(''.join, df_grouped.columns.values))

In [28]:
## Rename columns
df_grouped.columns = ['uf', 'setor_bndes', 'porte_do_cliente', 'ano', 'mean_valor_contratado_reais',
                 'mean_valor_desembolsado_reais', 'mean_juros', 'mean_prazo_carencia_meses', 
                 'mean_prazo_amortizacao_meses','mean_num_empregados', 'mean_emissao_co2', 'mean_pib']


In [30]:
## Round numeric values
toRound = ['mean_valor_contratado_reais','mean_valor_desembolsado_reais', 'mean_juros',
            'mean_prazo_carencia_meses', 'mean_prazo_amortizacao_meses', 'mean_num_empregados',
            'mean_emissao_co2', 'mean_pib']
df_grouped[toRound] = df_grouped[toRound].round(1)


In [31]:
## Exporting integrated dataset
df_grouped.to_csv('bases-auxiliares/operacoes-grouped-n1.csv',
              index=False, header=True)

### 3.4.2 - Agrupamento para os dados SEEG - PIB

In [32]:
## Drop porte_do_cliente
df_fin.drop('porte_do_cliente', axis = 1, inplace = True)

In [33]:
## Group dataset with mean values and by uf, setor_bndes, ano
df_grouped = df_fin.groupby(['uf', 'setor_bndes', 'ano'], as_index=False).agg(
    {'valor_contratado_reais': ['mean'], 'valor_desembolsado_reais': ['mean'],
     'juros': ['mean'], 'prazo_carencia_meses': ['mean'], 'prazo_amortizacao_meses': ['mean'],
     'num_empregados': ['mean'], 'emissao_co2': ['mean'], 'pib': ['mean']})

In [34]:
## Melt multi index from pandas
df_grouped.columns = list(map(''.join, df_grouped.columns.values))

In [36]:
## Rename columns
df_grouped.columns = ['uf', 'setor_bndes', 'ano', 'mean_valor_contratado_reais',
                      'mean_valor_desembolsado_reais', 'mean_juros', 'mean_prazo_carencia_meses',
                      'mean_prazo_amortizacao_meses', 'mean_num_empregados', 'mean_emissao_co2', 'mean_pib']

In [37]:
## Round numeric values
toRound = ['mean_valor_contratado_reais', 'mean_valor_desembolsado_reais', 'mean_juros',
           'mean_prazo_carencia_meses', 'mean_prazo_amortizacao_meses', 'mean_num_empregados',
           'mean_emissao_co2', 'mean_pib']
df_grouped[toRound] = df_grouped[toRound].round(1)

In [39]:
## Exporting integrated dataset
df_grouped.to_csv('bases-auxiliares/operacoes-grouped-n2.csv',
                  index=False, header=True)